# Cell detection visualization (with raw data)

### Simple tutorial to overlay raw lightsheet images with cells detected using ClearMap.

Depends on [ClearMapCluster](https://github.com/PrincetonUniversity/ClearMapCluster) installation and processed lightsheet images acquired using the LaVision UltraMicroscope.

In [1]:
#import modules
import numpy as np, os, time, cv2, pandas as pd
from skimage.external import tifffile
import matplotlib.pyplot as plt

Define key functions:

In [7]:
def resize_merged_stack(pth, dst, dtype = "uint16", resizef = 3):
    """
    resize function for large image stacks using cv2
    inputs:
        pth = 4d stack, memmap array or numpy array
        dst = path of tif file to save
        dtype = default uint16
        resizef = default 6
    """

    #read file
    if pth[-4:] == ".tif": img = tifffile.imread(pth)
    elif pth[-4:] == ".npy": img = np.lib.format.open_memmap(pth, dtype = dtype, mode = "r")
    else: img = pth #if array was input

    z,y,x,ch = img.shape
    resz_img = np.zeros((z, int(y/resizef), int(x/resizef), ch))

    for i in range(z):
        for j in range(ch):
            #make the factors - have to resize both image and cell center array
            xr = int(img[i, :, :, j].shape[1] / resizef); yr =  int(img[i, :, :, j].shape[0] / resizef)
            im = cv2.resize(img[i, :, :, j], (xr, yr), interpolation=cv2.INTER_LINEAR)
            resz_img[i, :, :, j] = im.astype(dtype)

    tifffile.imsave(dst, resz_img.astype(dtype))

    return dst

def check_cell_center_to_fullsizedata(brain, zstart, zstop, dst, resizef, annotation=False):
    """
    maps cnn cell center coordinates to full size cell channel images
    inputs:
        brain = path to lightsheet processed directory
        zstart = beginning of zslice
        zstop = end of zslice
        dst = path of tif stack to save
    NOTE: 20+ PLANES CAN OVERLOAD MEMORY
    """
    print(os.path.basename(brain))
    start = time.time()

    #doing things without loading parameter dict
    fzfld = os.path.join(brain, "full_sizedatafld")

    #exception if only 1 channel is imaged
    cellch = os.path.join(fzfld, [xx for xx in os.listdir(fzfld) if "647" in xx][0])

    #not the greatest way to do things, but works
    src = [os.path.join(cellch, xx) for xx in os.listdir(cellch) if xx[-3:] == "tif" and int(xx[-7:-4]) in range(zstart, zstop)]; src.sort()

    raw = np.zeros((len(src), tifffile.imread(src[0]).shape[0], tifffile.imread(src[0]).shape[1]))

    for i in range(len(src)):
        raw[i, :, :] = tifffile.imread(src[i])

    pth = os.path.join(brain, "clearmap_cluster_output/cells.npy")
    cells = np.load(pth) #this is in x,y,z!!!!!!!!!!!!!!!

    cells = cells[(cells[:, 2] >= zstart) & (cells[:, 2] <= zstop-1)] #-1 to account for range

    cell_centers = np.zeros(raw.shape)
    print("\n******making the corresponding cell map******\n")
    for i, r in enumerate(cells):
        cell_centers[r[2]-zstart, r[1]-1:r[1]+1, r[0]-1:r[0]+1] = 50000

    rbg = np.stack([raw.astype("uint16"), cell_centers.astype("uint16"), np.zeros_like(raw)], -1)
    #add the annotation volume transformed-to-raw-space as the 'blue' channel in the RGB stack (3 color overlay)
    if annotation:
        print("\n******making the annotation map******\n")
        annotation = os.path.join(annotation, "transformed_annotations/single_tifs") #the directory structure of these annotation volumes
        src = [os.path.join(annotation, xx) for xx in os.listdir(annotation) if xx[-3:] == "tif" and int(xx[-7:-4]) in range(zstart, zstop)]; src.sort()
        #populate corresponding annotation volume
        ann_raw = np.zeros((len(src), tifffile.imread(src[0]).shape[0], tifffile.imread(src[0]).shape[1]))
        for i in range(len(src)):
            ann_raw[i, :, :] = tifffile.imread(src[i])
        #add 'blue' channel to rbg stack
        rbg = np.stack([raw.astype("uint16"), cell_centers.astype("uint16"), ann_raw.astype("uint16")], -1)
        #FIXME: i am converting the annotation images to a 16 bit file. if the original annotation volume was 32 bit,
        #this can cause problems with the out of range structures. either use only 16 bit annotation volumes
        #for this or try something else
        
    print("\n******resizing (if requested)******\n")
    resize_merged_stack(rbg, os.path.join(dst, "{}_raw_cell_centers_resizedfactor{}_z{}-{}.tif".format(os.path.basename(brain),
                                          resizef, zstart, zstop)), "uint16", resizef)

    print("took %0.1f seconds to make merged maps for %s" % ((time.time()-start), brain))

def check_cell_center_to_resampled(brain, zstart, zstop, dst):
    """
    maps cnn cell center coordinates to resampled stack
    inputs:
        brain = path to lightsheet processed directory
        zstart = beginning of zslice
        zstop = end of zslice
        dst = path of tif stack to save
    NOTE: 20+ PLANES CAN OVERLOAD MEMORY
    """
    print(os.path.basename(brain))
    start = time.time()

    #doing things without loading parameter dict, could become a problem
    tifs = [xx for xx in os.listdir(brain) if xx[-4:] == ".tif"]; tifs.sort()
    raw = tifffile.imread(tifs[len(tifs)-1])

    pth = os.path.join(brain, "clearmap_cluster_output/cells.npy")
    cells = np.load(pth) #this is in x,y,z!!!!!!!!!!!!!!!

    cells = cells[(cells[:, 2] >= zstart) & (cells[:, 2] <= zstop-1)] #-1 to account for range

    cell_centers = np.zeros(raw.shape)
    print("\n******making the corresponding cell map******\n")
    for i, r in enumerate(cells):
        cell_centers[r[2]-zstart, r[1]-1:r[1]+1, r[0]-1:r[0]+1] = 50000

    rbg = np.stack([raw.astype("uint16"), cell_centers.astype("uint16"), np.zeros_like(raw)], -1)
    print("\n******resizing******\n")
    resize_merged_stack(rbg, os.path.join(dst, "{}_raw_cell_centers_resized_z{}-{}.tif".format(os.path.basename(brain),
                                          zstart, zstop)), "uint16", 6)

    print("took %0.1f seconds to make merged maps for %s" % ((time.time()-start), brain))

Set paths to source directory (where the ClearMapCluster processed directory exists) and destination directory (where you want to save it):

### Overlay annotations (optional)

If you have transformed your annotation volume corresponding the atlas to "raw" space (aka your full-resolution images), you can optionally specify the location of the transformed annotations by animal name (`annotation`). These transformations are typically done using `transformix` in the `elastix` package and should be done prior to running this notebook (see Z.D.'s [helper scripts](https://github.com/PrincetonUniversity/lightsheet_helper_scripts/blob/master/registration/annotation_volume_analysis/transform_annotations_to_fullsize_cfos.py) for an example on how to do this).

In [9]:
src = "/jukebox/LightSheetData/falkner-mouse/scooter/clearmap_processed"
dst = "/jukebox/LightSheetData/falkner-mouse/scooter/qc"
annotation = "/jukebox/LightSheetData/falkner-mouse/scooter/qc/annotation_volumes_in_raw_space"
#make destination directory if it does not already exist
if not os.path.exists(dst): os.mkdir(dst)

### Parameters for visualization

Raw lightsheet images are typically 2000 x 2000 pixels in size with ~700-1500 z-planes. Thus, the whole brain volume with an overlay of cells detected cannot be visualized with this method at once. You can, however, select a range of z-planes at a time so the script can execute and output a reasonably-sized volume to visualize on a standard desktop or laptop. 

As a reference, `z=0,1,2,...` represents the plane of the brain volume based on the orientation of imaging. If you image from dorsal --> ventral horizontally, `z=0` represents the dorsal-most area of the brain, and so on. 

You can set a `zstart` and `zstop` value which represents the number of planes you want to visualize at a time. To convert the # of planes into microns, you can simply multiple the number of planes you are visualizing by the z-step you selected during imaging. In this case, 20 planes = 20 * 10 micron z-step = 200 microns

#### Warning: typically you would not want to do more than 20 planes to run this on a standard desktop.

In [4]:
zstart = 400; zstop = 420 #z-plane #'s you want to visualize at a time, 250-400 is probably ideal for thalamus

You can also set a `resizef` as a factor by which you want to downsize your raw data and cell centers accordingly to visualize the overlay. You can do this if your machine has memory constraints, but typically visualizing the cell centers on the raw image is ideal as this is the resolution in which the cells were detected originally.

In [5]:
resizef = 1 #factor by which to downsize raw and cell center overlay, keep 1 if you do not want to downsize

### Run the overlay function

You can pick the names of brains/animals in your processed directory for which you want to make the cell center overlay, and execute the main `check_cell_center_to_fullsizedata()` iteratively:

In [10]:
ids = ["fmnp4"]

for i in ids:
    brain = os.path.join(src, i)
    annotation = os.path.join(annotation, i)
    check_cell_center_to_fullsizedata(brain, zstart, zstop, dst, resizef, annotation=annotation)

fmnp4

******making the corresponding cell map******


******making the annotation map******


******resizing (if requested)******



/home/wanglab/anaconda3/envs/lightsheet/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':
/home/wanglab/anaconda3/envs/lightsheet/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]


took 93.5 seconds to make merged maps for /jukebox/LightSheetData/falkner-mouse/scooter/clearmap_processed/fmnp4


You can also create overlays using the downsized whole-brain volume already created in the ClearMapCluster pipeline for registration, which saves an extra step of downsizing and may be friendlier for a laptop run:

In [ ]:
ids = ["fmnp4"]

for i in ids:
    brain = os.path.join(src, i)
    check_cell_center_to_resampled(brain, zstart, zstop, dst, resizef)